In [18]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
train = pd.read_csv('feature_hero_dcdd_riva_teama.csv')
train_x = train.drop(['1 - DUO_CARRY','1 - DUO_SUPPORT','1 - JUNGLE','1 - MID','1 - TOP',\
                      '2 - DUO_CARRY','2 - DUO_SUPPORT','2 - JUNGLE','2 - MID','2 - TOP',\
                      'carry_vs','support_vs','jungle_vs','mid_vs','top_vs',\
                      'teammates_jungle_mid','teammates_carry_sup','T1 win'],axis=1)
train_y = train[['T1 win']]
del train
print('train_x shape:',train_x.shape)
print('train_y shape:',train_y.shape)

train_x shape: (148634, 76)
train_y shape: (148634, 1)


In [19]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

lgb_train_x, lgb_valid_x, lgb_train_y, lgb_valid_y = train_test_split(train_x, train_y, test_size=0.3, random_state=0)

model_lgb = LGBMClassifier(boosting_type='gbdt', objective='binary', silent=True)

model_lgb.fit(lgb_train_x, lgb_train_y, 
                  eval_names=['validation'],
                  eval_metric=['logloss','auc'],
                  eval_set=[(lgb_valid_x, lgb_valid_y)],
                  early_stopping_rounds=10)

del lgb_train_x, lgb_valid_x, lgb_train_y, lgb_valid_y

[1]	validation's auc: 0.976059	validation's binary_logloss: 0.617757
Training until validation scores don't improve for 10 rounds
[2]	validation's auc: 0.97966	validation's binary_logloss: 0.556308
[3]	validation's auc: 0.980392	validation's binary_logloss: 0.504913
[4]	validation's auc: 0.98087	validation's binary_logloss: 0.461694
[5]	validation's auc: 0.981648	validation's binary_logloss: 0.424754
[6]	validation's auc: 0.9821	validation's binary_logloss: 0.392929
[7]	validation's auc: 0.982362	validation's binary_logloss: 0.36559
[8]	validation's auc: 0.982626	validation's binary_logloss: 0.341694
[9]	validation's auc: 0.982732	validation's binary_logloss: 0.320935
[10]	validation's auc: 0.982769	validation's binary_logloss: 0.302842
[11]	validation's auc: 0.982919	validation's binary_logloss: 0.286758
[12]	validation's auc: 0.983032	validation's binary_logloss: 0.272583
[13]	validation's auc: 0.983253	validation's binary_logloss: 0.260126
[14]	validation's auc: 0.983513	validation'